In [0]:
import pandas as pd
import numpy as np

from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures


In [3]:
!wget https://datahack-prod.s3.amazonaws.com/train_file/train_NIR5Yl1.csv

--2019-02-10 17:33:46--  https://datahack-prod.s3.amazonaws.com/train_file/train_NIR5Yl1.csv
Resolving datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)... 52.219.64.44
Connecting to datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)|52.219.64.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12564851 (12M) [application/vnd.ms-excel]
Saving to: ‘train_NIR5Yl1.csv’

train_NIR5Yl1.csv   100%[===================>]  11.98M  5.19MB/s    in 2.3s    

2019-02-10 17:33:49 (5.19 MB/s) - ‘train_NIR5Yl1.csv’ saved [12564851/12564851]



In [4]:
!wget https://datahack-prod.s3.amazonaws.com/test_file/test_8i3B3FC.csv

--2019-02-10 17:34:02--  https://datahack-prod.s3.amazonaws.com/test_file/test_8i3B3FC.csv
Resolving datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)... 52.219.64.32
Connecting to datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)|52.219.64.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4673330 (4.5M) [application/vnd.ms-excel]
Saving to: ‘test_8i3B3FC.csv’

test_8i3B3FC.csv    100%[===================>]   4.46M  1.91MB/s    in 2.3s    

2019-02-10 17:34:06 (1.91 MB/s) - ‘test_8i3B3FC.csv’ saved [4673330/4673330]



In [5]:
!unzip train_NIR5Yl1.csv
!unzip test_8i3B3FC.csv

Archive:  train_NIR5Yl1.csv
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of train_NIR5Yl1.csv or
        train_NIR5Yl1.csv.zip, and cannot find train_NIR5Yl1.csv.ZIP, period.
Archive:  test_8i3B3FC.csv
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of test_8i3B3FC.csv or
        test_8i3B3FC.csv.zip, and cannot find test_8i3B3FC.csv.ZIP, period.


In [0]:
train = pd.read_csv('train_NIR5Yl1.csv')
train = train.drop(train[train.Views > 3000000].index)

     
labelencoder_X = LabelEncoder()
train['Tag'] = labelencoder_X.fit_transform(train['Tag'])
train.drop(['ID','Username'], axis=1,inplace =True)
target = train['Upvotes']



In [0]:
from sklearn.preprocessing import Binarizer
bn = Binarizer(threshold=7)
pd_watched = bn.transform([train['Answers']])[0]
train['pd_watched'] = pd_watched




In [9]:
feature_names = [x for x in train.columns if x not in ['Upvotes']]
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(train[feature_names], target,test_size = 0.22,random_state =205)
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_val = sc_X.transform(x_val)



/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [10]:
poly_reg = PolynomialFeatures(degree = 4,interaction_only=False, include_bias=True)
X_poly = poly_reg.fit_transform(x_train)
poly_reg.fit(x_train, y_train)
lin_reg_1 = linear_model.LassoLars(alpha=0.021,max_iter=150)
lin_reg_1.fit(X_poly, y_train)



LassoLars(alpha=0.021, copy_X=True, eps=2.220446049250313e-16,
     fit_intercept=True, fit_path=True, max_iter=150, normalize=True,
     positive=False, precompute='auto', verbose=False)

In [11]:
# predicitng 
pred_val = lin_reg_1.predict(poly_reg.fit_transform(x_val))

print(r2_score(y_val, pred_val))



0.903437879332795


In [0]:
# ---------------------------------------------------------------------------------------

# testing

test = pd.read_csv('test_8i3B3FC.csv')
ids = test['ID']
test.drop(['ID','Username'], axis=1,inplace =True)


labelencoder_X = LabelEncoder()
test['Tag'] = labelencoder_X.fit_transform(test['Tag'])



In [13]:
from sklearn.preprocessing import Binarizer
bn = Binarizer(threshold=7)
pd_watched = bn.transform([test['Answers']])[0]
test['pd_watched'] = pd_watched

   
test = sc_X.fit_transform(test)

pred_test = lin_reg_1.predict(poly_reg.fit_transform(test))
pred_test=abs(pred_test)




/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [0]:
submission = pd.DataFrame({'ID': ids,
                           'Upvotes':pred_test
                           })

submission.to_csv("final_sub477.csv",index=False)

In [0]:
from google.colab import files
files.download("final_sub477.csv")